In [28]:
!pip install -q -r requirements.txt
!pre-commit autoupdate
!pre-commit install

[https://github.com/psf/black] already up to date!
[https://github.com/pycqa/isort] already up to date!
[https://github.com/PyCQA/flake8] already up to date!
[https://github.com/pre-commit/mirrors-mypy] already up to date!
pre-commit installed at .git/hooks/pre-commit


In [2]:
from src.data.utils import download_dataset


download_dataset()


Dataset found at dataset/stead
